In [9]:
import torch
import torch.utils.data
from torch.utils.data import TensorDataset, DataLoader
import argparse
import time
import gc
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from tqdm.notebook import tqdm
import os
import h5py
import numpy as np
from loader import ICUVariableLengthLoaderTables, ICUVariableLengthDataset ## from HiRID code
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, matthews_corrcoef

In [13]:
from net_dev import GNNStack

In [14]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [1]:
args = {
    'arch': 'dyGIN2d', 
    'dataset': 'hirid_dae',
    'num_layers': 2,  # the number of GNN layers  3
    'groups': 32,  # the number of time series groups (num_graphs)
    'pool_ratio': 0.0,  # the ratio of pooling for nodes # initially 0.1 but changed to 0 because the node number was decreasing
    'kern_size': [3,3],  # list of time conv kernel size for each layer [9,5,3]
    'in_dim': 64,  # input dimensions of GNN stacks
    'hidden_dim': 64,  # hidden dimensions of GNN stacks
    'out_dim': 64,  # output dimensions of GNN stacks
    'workers': 0,  # number of data loading workers
    'epochs': 20,  # number of total epochs to run
    'batch_size': 8,  # mini-batch size, this is the total batch size of all GPUs
    'val_batch_size': 8,  # validation batch size
    'lr': 0.0002,  # initial learning rate
    'weight_decay': 1e-4,  # weight decay
    'evaluate': False,  # evaluate model on validation set
    'seed': 3,  # seed for initializing training
    'gpu': 0,  # GPU id to use
    'use_benchmark': True,  # use benchmark
    'tag': 'date',  # the tag for identifying the log and model files
    'loss':'bce',
    'resample':'5min',
    'series_length':2016,
}

In [2]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    
    
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
              
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        
        # print(output, target)

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            try:
                res.append(correct_k.mul_(100.0 / batch_size))
            except:
                res.append(0)

        return res

def log_msg(message, log_file):
    with open(log_file, 'a') as f:
        print(message, file=f)


def get_default_train_val_test_loader(args):

    # get dataset-id
    dsid = args['dataset']
    h5_path= '<path to the HiRID h5 dataset>'
    
    num_nodes = 210
    seq_length = 2016
    num_classes = 2 
    
    
    task   = 'Dynamic_RespFailure_12Hours' ##27249
    maxlen            = 2016 #288 #2016                                                                         #|stays|      #-1 =8065                    #2016                      #difference
    data_loader_train = ICUVariableLengthDataset(source_path=h5_path, maxlen=maxlen, task=task, split='train') #| 19092  |    5646423(3467116+2179307)     4776429(2901908+1874521)   869994(565208+304786)
    data_loader_val   = ICUVariableLengthDataset(source_path=h5_path, maxlen=maxlen, task=task, split='val')   #| 4081   |    1220285(758822+461463)       1006329(614326+392003)     213956(144496+69460)
    data_loader_test  = ICUVariableLengthDataset(source_path=h5_path, maxlen=maxlen, task=task, split='test')  #| 4076   |    1209724(753935+455789)       1015139(620532+394607)     194585(133403+61182)
    
    train_loader = DataLoader(data_loader_train, batch_size=4, shuffle=True, num_workers=1,pin_memory=True, prefetch_factor=2)
    val_loader   = DataLoader(data_loader_val,   batch_size=4, shuffle=False, num_workers=1,pin_memory=True, prefetch_factor=2)
    # val_loader  = DataLoader(data_loader_test,  batch_size=4, shuffle=False, num_workers=1,pin_memory=True, prefetch_factor=2)
    test_loader  = DataLoader(data_loader_test,  batch_size=4, shuffle=False, num_workers=1,pin_memory=True, prefetch_factor=2)


    # data_loader
    # train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=args['batch_size'],shuffle=True,num_workers=args['workers'], pin_memory=True)
    # val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=args['val_batch_size'],shuffle=False,num_workers=args['workers'],pin_memory=True)


    return train_loader, val_loader, test_loader, num_nodes, seq_length, num_classes

In [17]:
featvent = [163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175]
featairy = [14,15,16,17,18,19]
feat_pf  = [54,187]

In [18]:
featjoined = featairy + featvent + feat_pf

In [19]:
len(featjoined)

21

In [ ]:
binary_array_210 = [1] * 231

# Set specified indexes to 0
indexes_to_set_zero = featjoined #[14, 15, 16, 17, 18, 19, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175]
for index in indexes_to_set_zero:
    binary_array_210[index] = 0


In [25]:
resp_select_bin= binary_array_210



In [26]:
# Convert the list to a numpy array
binary_array = np.array(resp_select_bin)
# Use numpy to find the indexes of the element 1
indexes_of_1_np = np.where(binary_array == 1)[0]
yes_index = indexes_of_1_np


# Convert the numpy array of indexes to a torch tensor
indexes_of_1_torch = torch.tensor(yes_index, dtype=torch.long)

In [29]:
def main_work(args):
    
    random.seed(args['seed'])
    torch.manual_seed(args['seed'])
 
    
    # init metrics
    best_acc1 = 0
    best_roc  = 0
    best_pr   = 0
    best_f1   = 0
    best_mcc  = 0
    
    best_test_acc1 = 0
    best_test_roc  = 0
    best_test_pr   = 0
    best_test_f1   = 0
    best_test_mcc  = 0    
    
    if args['tag'] == 'date':
        local_date = time.strftime('%m.%d %H:%M', time.localtime(time.time()))
        args['tag'] = local_date
        
    # Use the 'tag' which now contains either the date or a custom tag along with the dataset name for the directory
    run_dir_name = f"{args['dataset']}_{args['tag']}"
   
    # Base directory for saving models
    base_model_save_dir = "saved_models"
   
    # Specific directory for this run
    specific_model_save_dir = os.path.join(base_model_save_dir, run_dir_name)
    os.makedirs(specific_model_save_dir, exist_ok=True)

    print(f"Models will be saved in: {specific_model_save_dir}")        


    log_file = 'log/{}_gpu{}_{}_{}_exp.txt'.format(args['tag'], args['gpu'], args['arch'], args['dataset'])
    
    
    if args['gpu'] is not None:
        print("Use GPU: {} for training".format(args['gpu']))


    # dataset
    train_loader, val_loader, test_loader, num_nodes, seq_length, num_classes = get_default_train_val_test_loader(args)
    
    print('features / nodes', num_nodes,'total time graphs',args['groups'],'time series length',seq_length,'classes', num_classes)
    
    # training model from net.py
    model = GNNStack(gnn_model_type=args['arch'], num_layers=args['num_layers'], 
                     groups=args['groups'], pool_ratio=args['pool_ratio'], kern_size=args['kern_size'], 
                     in_dim=args['in_dim'], hidden_dim=args['hidden_dim'], out_dim=args['out_dim'], 
                     seq_len=seq_length, num_nodes=num_nodes, num_classes=num_classes)

    # print & log
    log_msg('epochs {}, lr {}, weight_decay {}'.format(args['epochs'], args['lr'], args['weight_decay']), log_file)
    
    log_msg(str(args), log_file)


    # determine whether GPU or not
    if not torch.cuda.is_available():
        print("Warning! Using CPU!!!")
    elif args['gpu'] is not None:
        torch.cuda.set_device(args['gpu'])

        # collect cache
        gc.collect()
        torch.cuda.empty_cache()

        model = model.cuda(args['gpu'])
        if args['use_benchmark']:
            cudnn.benchmark = True
        print('Using cudnn.benchmark.')
    else:
        print("Error! We only have one gpu!!!")
    
    criterion = nn.CrossEntropyLoss().cuda(args['gpu'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

    # validation
    if args['evaluate']:
        validate(val_loader, model, criterion, args)
        return

    # train & valid
    print('****************************************************')
    print('Dataset: ', args['dataset'])

    dataset_time = AverageMeter('Time', ':6.3f')

    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    epoches = []
    
    ########################
    loss_test = []
    acc_test  = []
    #######################
    
    ###### more lists to have values
    roc_train = []
    pr_train  = []
    f1_train  = []
    mcc_train = []
    
    roc_val   = []
    pr_val    = []
    f1_val    = []
    mcc_val   = []
    ##################################################
    roc_test   = []
    pr_test    = []
    f1_test    = []
    mcc_test   = []     
    #################################################
    end = time.time()
    for epoch in tqdm(range(args['epochs'])):
    # for epoch in range(args['epochs']):
        epoches += [epoch]

        # train for one epoch
        acc_train_per, loss_train_per, output_train_per, target_train_per = train(train_loader, model, criterion, optimizer, lr_scheduler, args)
        
        acc_train += [acc_train_per]
        loss_train += [loss_train_per]
        # calculate metric
        # print(len(target_train_per),len(output_train_per))
        auc_roc_value_train = roc_auc_score(target_train_per, output_train_per)
        auc_pr_value_train = average_precision_score(target_train_per, output_train_per)
        p2l_value_train = np.where(np.array(output_train_per) >= 0.5, 1, 0)
        f1_value_train = f1_score(target_train_per, p2l_value_train.tolist())
        mcc_value_train= matthews_corrcoef(target_train_per,p2l_value_train.tolist())
        
        #new code
        roc_train += [auc_roc_value_train]
        pr_train  += [auc_pr_value_train]
        f1_train  += [f1_value_train]
        mcc_train  += [mcc_value_train]

        msg = f'TRAIN, epoch {epoch}, train_loss {loss_train_per}, train_acc {acc_train_per}, train_roc {auc_roc_value_train:.5f}, \
                train_pr {auc_pr_value_train:.5f}, train_f1 {f1_value_train:.5f},train_mcc {mcc_value_train:.5f}'

        print(f'TRAIN, epoch {epoch}, train_loss {loss_train_per:.5f}, train_roc {auc_roc_value_train:.5f}, train_pr {auc_pr_value_train:.5f}, train_f1 {f1_value_train:.5f}, train_mcc {mcc_value_train:.5f}')
        # tqdm.write(f'TRAIN, epoch {epoch}, train_loss {loss_train_per:.5f}, train_roc {auc_roc_value_train:.5f}, train_pr {auc_pr_value_train:.5f}, train_f1 {f1_value_train:.5f}, train_mcc {mcc_value_train:.5f}')
        log_msg(msg, log_file)

        
        # evaluate on validation set
        acc_val_per, loss_val_per, output_val_per, target_val_per = validate(val_loader, model, criterion, args)

        acc_val  += [acc_val_per]
        loss_val += [loss_val_per]
        #calculate metric
        # calculate metric
        # print(len(target_val_per),len(output_val_per))
        auc_roc_value_val = roc_auc_score(target_val_per, output_val_per)
        auc_pr_value_val = average_precision_score(target_val_per, output_val_per)
        p2l_value_val = np.where(np.array(output_val_per) >= 0.5, 1, 0)
        f1_value_val = f1_score(target_val_per, p2l_value_val.tolist())
        mcc_value_val= matthews_corrcoef(target_val_per,p2l_value_val.tolist())
        #new code

        msg = f'VAL, epoch {epoch}, val_loss {loss_val_per}, val_acc {acc_val_per}, val_roc {auc_roc_value_val:.5f}, val_pr {auc_pr_value_val:.5f},val_f1 {f1_value_val:.5f}, val_mcc {mcc_value_val:.5f}'
        
        print(f'VAL, epoch {epoch}, val_loss {loss_val_per:.5f}, val_roc {auc_roc_value_val:.5f}, val_pr {auc_pr_value_val:.5f},val_f1 {f1_value_val:.5f}, val_mcc {mcc_value_val:.5f}')
        # tqdm.write(f'VAL, epoch {epoch}, val_loss {loss_val_per:.5f}, val_roc {auc_roc_value_val:.5f}, val_pr {auc_pr_value_val:.5f},val_f1 {f1_value_val:.5f}, val_mcc {mcc_value_val:.5f}')
        log_msg(msg, log_file)
        #########################################################################################################################
        # evaluate on test set
        acc_test_per, loss_test_per, output_test_per, target_test_per = validate(test_loader, model, criterion, args)

        acc_test   += [acc_test_per]
        loss_test  += [loss_test_per]
        #calculate metric
        # calculate metric
        # print(len(target_val_per),len(output_val_per))
        auc_roc_value_test = roc_auc_score(target_test_per, output_test_per)
        auc_pr_value_test  = average_precision_score(target_test_per, output_test_per)
        p2l_value_test = np.where(np.array(output_test_per) >= 0.5, 1, 0)
        f1_value_test = f1_score(target_test_per, p2l_value_test.tolist())
        mcc_value_test= matthews_corrcoef(target_test_per,p2l_value_test.tolist())
        #new code

        msg = f'TEST, epoch {epoch},test_loss {loss_test_per},test_acc {acc_test_per},test_roc {auc_roc_value_test:.5f},test_pr {auc_pr_value_test:.5f},test_f1 {f1_value_test:.5f}, test_mcc {mcc_value_test:.5f}'
        print(f'TEST, epoch {epoch}, test_loss {loss_test_per:.5f}, test_roc {auc_roc_value_test:.5f}, test_pr {auc_pr_value_test:.5f}, test_f1 {f1_value_test:.5f}, test_mcc {mcc_value_test:.5f}')
        # tqdm.write(f'TEST, epoch {epoch}, test_loss {loss_test_per:.5f}, test_roc {auc_roc_value_test:.5f}, test_pr {auc_pr_value_test:.5f}, test_f1 {f1_value_test:.5f}, test_mcc {mcc_value_test:.5f}')
        log_msg(msg, log_file)        
        
        #########################################################################################################################
        # remember best acc
        best_acc1 = max(acc_val_per, best_acc1)
        best_roc  = max(auc_roc_value_val, best_roc)
        best_pr   = max(auc_pr_value_val, best_pr)
        best_f1   = max(f1_value_val, best_f1)
        best_mcc   = max(mcc_value_val, best_mcc)
        
        #########################################################################################################################
        
        best_test_acc1 = max(acc_test_per, best_test_acc1)
        best_test_roc  = max(auc_roc_value_test, best_test_roc)
        best_test_pr   = max(auc_pr_value_test, best_test_pr)
        best_test_f1   = max(f1_value_test, best_test_f1)
        best_test_mcc   = max(mcc_value_test, best_test_mcc)
 

    #     wandb.log({"train_loss": loss_train_per, "train_roc": auc_roc_value_train, "train_pr": auc_pr_value_train, \
    #                "val_loss": loss_val_per, "val_roc": auc_roc_value_val, "val_pr": auc_pr_value_val, "best_val_roc": best_roc, "best_val_pr": best_pr})
    # wandb.finish()
    
    
    
        # Construct the filename with metrics
        auc_roc_value_val_scalar = auc_roc_value_val.item() if isinstance(auc_roc_value_val, torch.Tensor) else auc_roc_value_val
        auc_pr_value_val_scalar  = auc_pr_value_val.item() if isinstance(auc_pr_value_val, torch.Tensor) else auc_pr_value_val
        f1_value_val_scalar      = f1_value_val.item() if isinstance(f1_value_val, torch.Tensor) else f1_value_val
        mcc_value_val_scalar     = mcc_value_val.item() if isinstance(mcc_value_val, torch.Tensor) else mcc_value_val
        #########################################################################################################################
        auc_roc_value_test_scalar = auc_roc_value_test.item() if isinstance(auc_roc_value_test, torch.Tensor) else auc_roc_value_test
        auc_pr_value_test_scalar  = auc_pr_value_test.item() if isinstance(auc_pr_value_test, torch.Tensor) else auc_pr_value_test
        f1_value_test_scalar      = f1_value_test.item() if isinstance(f1_value_test, torch.Tensor) else f1_value_test
        mcc_value_test_scalar      = mcc_value_test.item() if isinstance(mcc_value_test, torch.Tensor) else mcc_value_test        
        #########################################################################################################################
        # Now use these scalar values in the filename
        filename = f"model_epoch_{epoch}.pth"
       
        # Continue with the existing logic to save the model
        model_path = os.path.join(specific_model_save_dir, filename)
        torch.save(model.state_dict(), model_path)

    # measure elapsed time
    dataset_time.update(time.time() - end)

    # log & print the best_acc
    msg = f'\n\n * BEST_ACC: {best_acc1}\n * TIME: {dataset_time}\n'
    log_msg(msg, log_file)

    print(f' * best_acc1: {best_acc1}, best_roc: {best_roc}, best_pr: {best_pr}, best_f1: {best_f1}, best_mcc: {best_mcc}')
    print(f' * best_test_acc1: {best_test_acc1}, best_test_roc: {best_test_roc}, best_test_pr: {best_test_pr}, best_test_f1: {best_test_f1}, best_test_mcc: {best_test_mcc}')
    print(f' * time: {dataset_time}')
    print('****************************************************')


    # collect cache
    gc.collect()
    torch.cuda.empty_cache()


def train(train_loader, model, criterion, optimizer, lr_scheduler, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    
    output_list = []
    target_list = [] 

    # switch to train mode
    model.train()

    for count, (data, label, mask) in enumerate(train_loader):

        # data in cuda
        
        ### data filter
        
        data_fil = torch.index_select(data, 2, indexes_of_1_torch) ## this selects the relevant features
        
        data = data_fil.cuda(args['gpu']).type(torch.float)
        data = data.permute(0, 2, 1).unsqueeze(1) ### this is correct
        
        mask = mask.cuda(args['gpu']).type(torch.bool)
        label = label.cuda(args['gpu']).type(torch.long)

        # compute output
        # output = model(data)
        output, g_const_adj, adj, adj_notopk = model(data)
        # print(len(output))
        # print('output', output.shape, 'mask', mask.shape)
        out_flat = torch.masked_select(output, mask.unsqueeze(-1)).reshape(-1, output.shape[-1])
        # print(output)
        # print(output.shape, mask.shape, out_flat.shape)
        # break
        # out_flat = torch.masked_select(output[:,:,1], mask)
        

        label_flat = torch.masked_select(label, mask)
        # print('output',output.shape, 'mask', mask.shape,'out_flat', out_flat.shape, 'label', label.shape, 'label_flat', label_flat.shape)
        loss = criterion(out_flat, label_flat)

        # measure accuracy and record loss
        acc1 = accuracy(out_flat, label_flat, topk=(1, 1))
        
        output_np = torch.softmax(out_flat, dim=1).detach().cpu().numpy()[:,1].tolist()
        
        target_np = label_flat.detach().cpu().numpy().tolist()
        
        # print(output_np, target_np)
        
        losses.update(loss.item(), data.size(0))
        top1.update(acc1[0], data.size(0))
        
        # met_roc.update(roc, data.size(0))
        # met_pr.update(pr, data.size(0))
        output_list += output_np
        target_list += target_np

        # compute gradient and do Adam step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    lr_scheduler.step(top1.avg)

    return top1.avg, losses.avg, output_list, target_list


def validate(val_loader, model, criterion, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    output_list = []
    target_list = [] 
    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for count, (data, label, mask) in enumerate(val_loader):
            
            data_fil = torch.index_select(data, 2, indexes_of_1_torch)  ## this selects the relevant features
        
            # data = data_fil.cuda(args['gpu']).type(torch.float)
            # data = data.cuda(args['gpu']).type(torch.float)
            
            
            
            if args['gpu'] is not None:
                
                data = data_fil.cuda(args['gpu'], non_blocking=True).type(torch.float)
                data = data.permute(0, 2, 1).unsqueeze(1)
            if torch.cuda.is_available():
                label = label.cuda(args['gpu'], non_blocking=True).type(torch.long)
                mask  = mask.cuda(args['gpu'], non_blocking=True).type(torch.bool)

            # compute output
            # output = model(data)
            output, g_const_adj, adj, adj_notopk = model(data)
            
            out_flat = torch.masked_select(output, mask.unsqueeze(-1)).reshape(-1, output.shape[-1])
            label_flat = torch.masked_select(label, mask)

            loss = criterion(out_flat, label_flat)
            
            output_np = torch.softmax(out_flat, dim=1).detach().cpu().numpy()[:,1].tolist()
            target_np = label_flat.detach().cpu().numpy().tolist()

            # measure accuracy and record loss
            acc1 = accuracy(out_flat, label_flat, topk=(1, 1))
            losses.update(loss.item(), data.size(0))
            top1.update(acc1[0], data.size(0))
            
            output_list += output_np
            target_list += target_np
            
            # met_roc.update(roc, data.size(0))
            # met_pr.update(pr, data.size(0))

    return top1.avg, losses.avg, output_list, target_list

In [30]:
main_work(args)

In [31]:
def load_model(model_path, model_class, *model_args, **model_kwargs):
    """
    Load the model from a saved state dictionary.

    Parameters:
    - model_path: Path to the saved model state dictionary.
    - model_class: The class of the model to instantiate.
    - model_args: Positional arguments for the model class instantiation.
    - model_kwargs: Keyword arguments for the model class instantiation.

    Returns:
    - model: The loaded model ready for prediction.
    """
    # Instantiate the model
    model = model_class(*model_args, **model_kwargs)
    # Load the saved state dictionary
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode
    return model

def predict(test_loader, model, criterion, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    output_list = []
    target_list = [] 
    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for count, (data, label, mask) in enumerate(test_loader):
            
            data_fil = torch.index_select(data, 2, indexes_of_1_torch)  ## this selects the relevant features       

            if args['gpu'] is not None:
                
                data = data_fil.cuda(args['gpu'], non_blocking=True).type(torch.float)
                data = data.permute(0, 2, 1).unsqueeze(1)
            if torch.cuda.is_available():
                label = label.cuda(args['gpu'], non_blocking=True).type(torch.long)
                mask  = mask.cuda(args['gpu'], non_blocking=True).type(torch.bool)

            # compute output
            # output = model(data)
            output, g_const_adj, adj, adj_notopk  = model(data)
            
            out_flat = torch.masked_select(output, mask.unsqueeze(-1)).reshape(-1, output.shape[-1])
            label_flat = torch.masked_select(label, mask)

            loss = criterion(out_flat, label_flat)
            
            output_np = torch.softmax(out_flat, dim=1).detach().cpu().numpy()[:,1].tolist()
            target_np = label_flat.detach().cpu().numpy().tolist()

            # measure accuracy and record loss
            acc1 = accuracy(out_flat, label_flat, topk=(1, 1))
            losses.update(loss.item(), data.size(0))
            top1.update(acc1[0], data.size(0))
            
            output_list += output_np
            target_list += target_np
            # met_roc.update(roc, data.size(0))
            # met_pr.update(pr, data.size(0))

    return top1.avg, losses.avg, output_list, target_list, g_const_adj, adj,adj_notopk